In [2]:
%matplotlib inline
import os
import sys

from collections import OrderedDict
from copy import deepcopy
from time import time

import matplotlib.pyplot as plt
import numpy as np
import scipy.special as sc

import pymc3 as pm
import seaborn as sns
import theano
import theano.tensor as tt

from pymc3 import Dirichlet, Poisson, Gamma
from pymc3 import math as pmmath
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from theano import shared
from theano.sandbox.rng_mrg import MRG_RandomStreams

# unfortunately I was not able to run it on GPU due to overflow problems
%env THEANO_FLAGS=device=cpu,floatX=float64

plt.style.use("seaborn-darkgrid")

env: THEANO_FLAGS=device=cpu,floatX=float64


In [3]:
class LDAEncoder:
    """Encode (term-frequency) document vectors to variational means and (log-transformed) stds."""

    def __init__(self, n_words, n_hidden, n_topics, p_corruption=0, random_seed=1):
        rng = np.random.RandomState(random_seed)
        self.n_words = n_words
        self.n_hidden = n_hidden
        self.n_topics = n_topics
        self.w0 = shared(0.01 * rng.randn(n_words, n_hidden).ravel(), name="w0")
        self.b0 = shared(0.01 * rng.randn(n_hidden), name="b0")
        self.w1 = shared(0.01 * rng.randn(n_hidden, 2 * (n_topics)).ravel(), name="w1")
        self.b1 = shared(0.01 * rng.randn(2 * (n_topics)), name="b1")
        self.rng = MRG_RandomStreams(seed=random_seed)
        self.p_corruption = p_corruption

    def encode(self, xs):
        if 0 < self.p_corruption:
            dixs, vixs = xs.nonzero()
            mask = tt.set_subtensor(
                tt.zeros_like(xs)[dixs, vixs],
                self.rng.binomial(size=dixs.shape, n=1, p=1 - self.p_corruption),
            )
            xs_ = xs * mask
        else:
            xs_ = xs

        w0 = self.w0.reshape((self.n_words, self.n_hidden))
        w1 = self.w1.reshape((self.n_hidden, 2 * self.n_topics))
        hs = tt.tanh(xs_.dot(w0) + self.b0)
        zs = hs.dot(w1) + self.b1
        zs_mean = zs[:, :self.n_topics]
        zs_rho = zs[:, self.n_topics: ]
        return {"mu": zs_mean, "rho": zs_rho}

    def get_params(self):
        return [self.w0, self.b0, self.w1, self.b1]

In [4]:
n_words = 1000

print("Loading dataset...")
t0 = time()
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=("headers", "footers", "quotes"))
data_samples = dataset.data
print("done in %0.3fs." % (time() - t0))

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=n_words, stop_words="english")

t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
feature_names = tf_vectorizer.get_feature_names()
print("done in %0.3fs." % (time() - t0))

Loading dataset...
done in 1.258s.
Extracting tf features for LDA...
done in 1.407s.


In [5]:
n_samples_tr = 10000
n_samples_te = tf.shape[0] - n_samples_tr
docs_tr = tf[:n_samples_tr, :]
docs_te = tf[n_samples_tr:, :]
print("Number of docs for training = {}".format(docs_tr.shape[0]))
print("Number of docs for test = {}".format(docs_te.shape[0]))

n_tokens = np.sum(docs_tr[docs_tr.nonzero()])
print(f"Number of tokens in training set = {n_tokens}")
print(
    "Sparsity = {}".format(len(docs_tr.nonzero()[0]) / float(docs_tr.shape[0] * docs_tr.shape[1]))
)

Number of docs for training = 10000
Number of docs for test = 1314
Number of tokens in training set = 480280
Sparsity = 0.0253937


In [82]:
def logp_lda_doc(beta, theta, n):
    """Returns the log-likelihood function for given documents.

    K : number of topics in the model
    V : number of words (size of vocabulary)
    D : number of documents (in a mini-batch)

    Parameters
    ----------
    beta : tensor (K x V)
        Word distributions.
    theta : tensor (D x K)
        Topic distributions for documents.
    n: tensor (D x 1)
        Lengths of each documents
    """

    def ll_docs_f(docs):
        
        """
        \log p(d | theta, beta, n) = \sum_w \log p(w | theta, beta)
                                   = \sum_w \log Poisson(w | theta @ beta * n) 
                                   = \sum_w \log Poisson(w | \sum_k theta_k * beta_k)
                                   = \sum_w \log Poisson(w | \sum_k \exp( \log theta_k + \log beta_k ))
                                   = \sum_w - \sum_k \exp( \log theta_k + \log beta_k ) + w * \log \sum_k \exp( \log theta_k + \log beta_k ) - \log gamma(w + 1)
        """
        dixs, vixs = docs.nonzero()
        n = docs.sum(1)
        vfreqs = docs[dixs, vixs]
        ll_docs = (
            (theta[dixs] + beta.T[vixs]).sum(1) *  n[dixs] + 
            vfreqs * n[dixs] * pmmath.logsumexp(tt.log(theta[dixs]) + tt.log(beta.T[vixs]), axis=1).ravel() - \
            pm.distributions.special.gammaln(vfreqs + 1)
        )
        # Per-word log-likelihood times num of tokens in the whole dataset
        return tt.sum(ll_docs) / (tt.sum(vfreqs) + 1e-9) * n_tokens

    return ll_docs_f

In [85]:
n = Poisson.dist(
    mu=pm.floatX(avg_len) * np.ones((minibatch_size, 1)),
    shape=(minibatch_size, 1),
).random()

beta = Dirichlet.dist(
    a=pm.floatX((1.0 / n_topics) * np.ones((n_topics, n_words))),
    shape=(n_topics, n_words),
).random()

theta = Dirichlet.dist(
    a=pm.floatX((1.0 / n_topics) * np.ones((minibatch_size, n_topics))),
    shape=(minibatch_size, n_topics),
    # do not forget scaling
).random()

In [88]:
n_topics = 10
minibatch_size = 32
avg_len = docs_tr.sum(1).mean()

doc_t_minibatch = pm.Minibatch(docs_tr.toarray(), minibatch_size)
doc_t = shared(docs_tr.toarray()[:minibatch_size])


e0 = c0 = 1.
f0 = .01
pn = .5

with pm.Model() as model:
    n = Poisson(
        "n", 
        mu=pm.floatX(avg_len) * np.ones((minibatch_size, 1)),
        shape=(minibatch_size, 1),
        total_size=n_samples_tr
    )
    
    beta = Dirichlet(
        "beta",
        a=pm.floatX((1.0 / n_topics) * np.ones((n_topics, n_words))),
        shape=(n_topics, n_words),
    )

    theta = Dirichlet(
        "theta",
        a=pm.floatX((1.0 / n_topics) * np.ones((minibatch_size, n_topics))),
        shape=(minibatch_size, n_topics),
        # do not forget scaling
        total_size=n_samples_tr,
    )
    
    # Note, that we defined likelihood with scaling, so here we need no additional `total_size` kwarg
    doc = pm.DensityDist("doc", logp_lda_doc(beta, theta, n), observed=doc_t)

    step = pm.Metropolis()
    trace = pm.sample(1000, step)
    

/home/liutianc/miniconda3/lib/python3.7/site-packages/pymc3/data.py:246: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  self.shared = theano.shared(data[in_memory_slc])
/home/liutianc/miniconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
